In [2]:
using Turing
using Random
using ForwardDiff
using LimberJack
using CSV
using NPZ
using FITSIO
using Plots
using LinearAlgebra
using BlockArrays
using Distributed
using DataFrames

┌ Info: Precompiling LimberJack [6b86205d-155a-4b14-b82d-b6a149ea78f2]
└ @ Base loading.jl:1423
┌ Warning: Package LimberJack does not have NPZ in its dependencies:
│ - If you have LimberJack checked out for development and have
│   added NPZ as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LimberJack
└ Loading NPZ into LimberJack from project dependency, future warnings for LimberJack are suppressed.


In [3]:
# Add four processes to use for sampling.
addprocs(4)

# Initialize everything on all the processes.
# Note: Make sure to do this after you've already loaded Turing,
#       so each process does not have to precompile.
#       Parallel sampling may fail silently if you do not do this.
@everywhere using Turing, LimberJack

      From worker 4:	┌ Warning: Package LimberJack does not have NPZ in its dependencies:
      From worker 4:	│ - If you have LimberJack checked out for development and have
      From worker 4:	│   added NPZ as a dependency but haven't updated your primary
      From worker 4:	│   environment's manifest file, try `Pkg.resolve()`.
      From worker 4:	│ - Otherwise you may need to report an issue with LimberJack
      From worker 4:	└ Loading NPZ into LimberJack from project dependency, future warnings for LimberJack are suppressed.
      From worker 5:	┌ Warning: Package LimberJack does not have NPZ in its dependencies:
      From worker 5:	│ - If you have LimberJack checked out for development and have
      From worker 5:	│   added NPZ as a dependency but haven't updated your primary
      From worker 5:	│   environment's manifest file, try `Pkg.resolve()`.
      From worker 5:	│ - Otherwise you may need to report an issue with LimberJack
      From worker 5:	└ Loading NPZ into Lim

In [4]:
desgs_data = npzread("/home/jaime/PhD/LimberJack.jl/data/cl_DESgc__2_DESwl__3.npz")["cl"]
desgs_data = transpose(desgs_data)[1:39]
desgs_cov = npzread("/home/jaime/PhD/LimberJack.jl/data/cov_DESgc__2_DESwl__3_DESgc__2_DESwl__3.npz")["cov"]
desgs_cov = desgs_cov[1:39, 1:39]
desgs_cov = Symmetric(Hermitian(desgs_cov))
desgs_err = (view(desgs_cov, diagind(desgs_cov)))[1:39].^0.5
desgs_nzs = FITS("/home/jaime/PhD/LimberJack.jl/data/y1_redshift_distributions_v1.fits")
desgs_nz = read(desgs_nzs["nz_source_mcal"], "BIN2")
desgs_zs = read(desgs_nzs["nz_source_mcal"], "Z_MID");

desgg_data = npzread("/home/jaime/PhD/LimberJack.jl/data/cl_DESgc__2_DESgc__2.npz")["cl"]
desgg_data = transpose(desgg_data)[1:39]
desgg_cov = npzread("/home/jaime/PhD/LimberJack.jl/data/cov_DESgc__2_DESgc__2_DESgc__2_DESgc__2.npz")["cov"]
desgg_cov = desgg_cov[1:39, 1:39]
desgg_cov = Symmetric(Hermitian(desgg_cov))
desgg_err = (view(desgg_cov, diagind(desgg_cov)))[1:39].^0.5
desgg_nzs = FITS("/home/jaime/PhD/LimberJack.jl/data/y1_redshift_distributions_v1.fits")
desgg_nz = read(desgg_nzs["nz_source_mcal"], "BIN2")
desgg_zs = read(desgg_nzs["nz_source_mcal"], "Z_MID");

ell = npzread("/home/jaime/PhD/LimberJack.jl/data/cl_DESgc__2_DESwl__3.npz")["ell"]
ell = [Int(floor(l)) for l in ell]
path = "/home/jaime/PhD/LimberJack.jl/data/";

In [5]:
data = [desgg_data, desgs_data]
data = vcat(data...);

In [6]:
#cov_tot = BlockArray{Number}(undef_blocks, fill(39, 2), fill(39, 2))
#setblock!(cov_tot, desgg_cov, 1, 1)
#setblock!(cov_tot, zeros(39,39), 1, 2)
#setblock!(cov_tot, zeros(39,39), 2, 1)
#setblock!(cov_tot, desgs_cov, 2, 2)
#cov_tot = Symmetric(Hermitian(Matrix(cov_tot)))

In [39]:
k = 0 
for i in 1:3
    for j in 1:3
        k = 1+k
        println(k)
    end
end

1
2
3
4
5
6
7
8
9


In [44]:
dims = 3
len = 1
covs = [[1], [2], [3], [4], [5], [6], [7], [8], [9]]
cov_tot = zeros(len*dims,len*dims)
k = 0
for j in 1:dims
    for i in 1:dims
        k = 1+k
        for l in 1:len
            cov_tot[(len*(j-1))+l, (len*(i-1))+1:1:i*len] = covs[k][l, 1:len]
        end
    end
end
cov_tot

3×3 Matrix{Float64}:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

In [24]:
dims = 2
covs = [desgg_cov, zeros(39,39), zeros(39,39), desgs_cov]
cov_tot = zeros(39*dims,39*dims)
for j in 1:dims
    for i in 1:dims
        for l in 1:39
            println(j, l)
            cov_tot[(39*(j-1))+l, (39*(i-1))+1:1:i*39] = covs[j][l, 1:39]
        end
    end
end
cov_tot

11
12
13
14
15
16
17
18
19
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
11
12
13
14
15
16
17
18
19
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
21
22
23
24
25
26
27
28
29
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
21
22
23
24
25
26
27
28
29
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239


78×78 Matrix{Float64}:
  2.35957e-10  -1.64815e-11   3.28313e-13  …  -3.27087e-17  -2.93001e-17
 -1.64815e-11   5.49845e-11  -4.42578e-12     -2.20232e-17  -1.95444e-17
  3.28313e-13  -4.42578e-12   1.42587e-11     -8.592e-18    -7.55724e-18
 -4.15482e-13   3.23412e-14  -1.15993e-12     -5.06005e-18  -4.41447e-18
 -1.35344e-13  -1.54344e-13   2.62272e-16     -2.43724e-18  -2.10575e-18
 -9.21085e-14  -6.19178e-14  -4.2601e-14   …  -1.55055e-18  -1.32602e-18
 -6.14421e-14  -4.02005e-14  -1.79269e-14     -1.08004e-18  -9.10587e-19
 -3.64203e-14  -2.94106e-14  -1.21992e-14     -7.1147e-19   -5.91584e-19
 -2.91058e-14  -1.93819e-14  -8.89479e-15     -5.38293e-19  -4.41964e-19
 -2.26189e-14  -1.52657e-14  -6.0968e-15      -4.09e-19     -3.31286e-19
 -1.81295e-14  -1.23596e-14  -4.77608e-15  …  -3.05901e-19  -2.43998e-19
 -1.53817e-14  -9.63336e-15  -3.53146e-15     -2.41036e-19  -1.89159e-19
 -9.93396e-15  -6.89733e-15  -2.72509e-15     -1.92675e-19  -1.48655e-19
  ⋮                         

In [9]:
@model function DES_model(data)
    Ωm ~ Uniform(0.2, 0.3)
    h ~ Uniform(0.6, 0.8)
    s8 ~ Uniform(0.7, 1.0)
    cosmology = LimberJack.Cosmology(Ωm, 0.05, h, 0.96, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    tg = NumberCountsTracer(cosmology, desgg_zs, desgg_nz, 2.)
    ts = WeakLensingTracer(cosmology, desgs_zs, desgs_nz)
    gg = [angularCℓ(cosmology, tg, tg, ℓ) for ℓ in ell]
    gs = [angularCℓ(cosmology, tg, tg, ℓ) for ℓ in ell]
    predictions = [gg, gs]
    predictions = vcat(predictions...)
    data ~ MvNormal(predictions, cov_tot)
end;

iterations = 10
step_size = 0.05
samples_per_step = 5
cores = 4

# Start sampling.

folname = string("DES_cobmined_test_", "stpsz_", step_size, "_smpls_", samples_per_step)
if isdir(folname)
    println("Folder already exists")
    if isfile("chain.jls")
        println("Restarting from past chain")
        past_chain = read("chain.jls", Chains)
        new_chain = sample(DES_model(data), HMC(step_size, samples_per_step),
                           MCMCDistributed(), iterations, cores, progress=true;
                           save_state=true, resume_from=past_chain)
    end
else
    mkdir(folname)
    println("Created new folder")
    new_chain = sample(DES_model(data), HMC(step_size, samples_per_step),
                       MCMCDistributed(), iterations, cores, progress=true; save_state=true)
end

info = describe(new_chain)[1]
fname_info = string("info.csv")
CSV.write(joinpath(folname, fname_info), info)


fname_jls = string("chain.jls")
write(joinpath(folname, fname_jls), new_chain)
    
fname_csv = string("chain.csv")
CSV.write(joinpath(folname, fname_csv), new_chain)

Created new folder
      From worker 3:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 3:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 3:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 5:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 5:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 5:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 2:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 2:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 2:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 4:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 4:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false

Sampling (4 processes): 100%|███████████████████████████| Time: 0:00:11


      From worker 3:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 5:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 5:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 5:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 4:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 4:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 4:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 2:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 2:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 2:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 2:	┌ Warning: The current proposal will be rejected due to numerical

┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364


"DES_cobmined_test_stpsz_0.05_smpls_5/chain.csv"